# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

# Aggregating the amount spent for unique customers
money_customers = orders.groupby(['CustomerID', 'Country'], as_index = False)\
.agg({'amount_spent':'sum'})


# Using the Binning system
labels = ['Random Customer', 'Regular Customer', 'Preferred Customer', 'VIP Customer']
cutoffs = [0, 0.50, 0.75, 0.95, 1]
binning = pd.qcut(money_customers['amount_spent'], cutoffs, labels = labels)


# Merging to display it all together
money_customers = pd.merge(money_customers, binning, left_index = True, right_index = True)
money_customers.reset_index(drop = True, inplace = True)

# Renaming columns to make it prettier
money_customers.columns = ['Customer ID', 'Country', 'Amount Spent', 'Type of Customer']
money_customers

,Customer ID,Country,Amount Spent,Type of Customer
0,12346,United Kingdom,77183.60,VIP Customer
1,12347,Iceland,4310.00,Preferred Customer
2,12348,Finland,1797.24,Preferred Customer
3,12349,Italy,1757.55,Preferred Customer
4,12350,Norway,334.40,Random Customer
...,...,...,...,...
4342,18280,United Kingdom,180.60,Random Customer
4343,18281,United Kingdom,80.82,Random Customer
4344,18282,United Kingdom,178.05,Random Customer
4345,18283,United Kingdom,2094.88,Preferred Customer


In [4]:
# Checking how many types of customers we have in each bin
money_customers['Type of Customer'].value_counts()

Random Customer       2174
Regular Customer      1086
Preferred Customer     869
VIP Customer           218
Name: Type of Customer, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [5]:
# your code here
# I do not understand this, tried getting marie's code for help and changed a few things
# but it is still not running

# I don't know why I keep getting key error on customer id, tried changing to all possible ways lol
country_customer = orders.groupby(['Customer ID', 'Country'], as_index = False).sum()#.reset_index(inplace = True)
country_customer.reset_index(inplace = True)

# merge countries with labels on CustomerID
c_a_merge = country_customer.merge(money_customers, on = 'Customer ID')

#select the country with the most VIPs
#select only VIP, groupby Country, count labels
VIP = c_a_merge[c_a_merge['Type of Customer'] == 'VIP Customer']
VIP_country = VIP.groupby(by = 'Country').count()['Type of Customer']

# sort the values, select index 0
VIP_country = VIP_country.sort_values(ascending = False).index[0]

VIP_country

# I give up on this way, cannot understand and make it work. Tried other things below, probably not 100%
# right, but at least got some output. Help will be much appreciated.

KeyError: 'Customer ID'

In [6]:
# Did this with Beatriz. Maybe not perfect, but it works?

# This shows us all the countries where there are 'VIP Customer'
filtered = money_customers[money_customers["Type of Customer"] == "VIP Customer"]
filtered

# This seems to do the count to get the highest country with VIPs? Strange all 3 values are the same?
count_ = pd.DataFrame(filtered.groupby("Country").count()).sort_values('Type of Customer', \
                                                                         ascending = False)
count_

,Customer ID,Amount Spent,Type of Customer
Country,,,
United Kingdom,178,178,178
Germany,11,11,11
France,9,9,9
Switzerland,3,3,3
Spain,2,2,2
Portugal,2,2,2
Japan,2,2,2
EIRE,2,2,2
Finland,1,1,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [7]:
# your code here
# I don't get why this isn't working. The condition is right, I believe?
filtered_2 = money_customers[(money_customers["Type of Customer"] == "VIP Customer") & 
                           (money_customers["Type of Customer"] == "Preferred Customer")]
filtered_2

,Customer ID,Country,Amount Spent,Type of Customer
